### Libraires

In [2]:
import sklearn
#!pip install plotly --user
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import pandas as pd
import numpy as np
import sqlite3
import textman as tx
from gensim.models import word2vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [3]:
db_file='./novels.db'

In [4]:
OHCO = ['book_num','chap_num','para_num','sent_num']
with sqlite3.connect(db_file) as db:
     tokens=pd.read_sql("SELECT * FROM token", db)
tokens = tokens[~tokens.term_str.isna()]
corpus = tokens.groupby(OHCO).term_str.apply(lambda  x:  x.tolist())\
    .reset_index()['term_str'].tolist()

### Generate word embeddings with Gensim's library

In [5]:
model = word2vec.Word2Vec(corpus, size=246, window=5, min_count=200, workers=4)

### Visualize with tSNE

In [6]:
coords = pd.DataFrame(index=range(len(model.wv.vocab)))
coords['label'] = [w for w in model.wv.vocab]
coords['vector'] = coords['label'].apply(lambda x: model.wv.get_vector(x))

In [7]:
tsne_model = TSNE(perplexity=40, n_components=3, init='pca', n_iter=2500, random_state=23)
tsne_values = tsne_model.fit_transform(coords['vector'].tolist())

In [8]:
coords['x'] = tsne_values[:,0]
coords['y'] = tsne_values[:,1]
coords["z"]=tsne_values[:,2]

In [9]:
coords.head()

,label,vector,x,y,z
0,the,"[-0.5734609, -1.0074835, -0.9610861, -0.076018...",-20.024199,-8.307119,-6.322312
1,heart,"[-1.3887401, -1.9154981, -1.6126525, -0.811983...",18.536423,-22.144844,10.668086
2,of,"[-0.015454118, -0.1963343, -1.8105001, -0.1874...",1.231350,-2.020259,-11.925744
3,rome,"[-0.29846457, -0.584723, 0.25645918, 0.4676312...",-13.155907,-3.202682,-18.767643
4,i,"[0.11904274, 0.42246172, -0.10948593, 0.049479...",41.820728,10.979006,-4.317355


In [10]:
df=coords
tools.set_credentials_file(username='nh4mq', api_key='dSgtevzVSngapiW6qj3I')

scatter = dict(
    mode = "markers",
    name = "y",
    type = "scatter3d",    
    x = df['x'], y = df['y'], z = df['z'],
    text = df["label"],
    marker = dict( size=2, color=df["z"],                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8)
)
layout = dict(
    title = '3d point TSNE',
    scene = dict(
        xaxis = dict( zeroline=False ),
        yaxis = dict( zeroline=False ),
        zaxis = dict( zeroline=False ),
    )
)
fig = dict( data=[scatter], layout=layout )
# Use py.iplot() for IPython notebook
py.iplot(fig, filename='3d point TSNE')

